In [2]:
# Import Python standard library and IPython packages we need.
import os
import subprocess
import sys
import csv
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from pprint import pprint
import json
import time
import requests
from requests.auth import HTTPBasicAuth

# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
os.environ["ACTINIA_USER"] = 'actinia-gdi'
os.environ["ACTINIA_PASSWORD"] = 'actinia-gdi'
os.environ["AUTH"] = 'actinia-gdi:actinia-gdi'
os.environ["ACTINIA_URL"] = 'http://localhost:8088'

ACTINIA_VERSION = 'v3'
ACTINIA_BASEURL = 'http://localhost:8088'
ACTINIA_URL = ACTINIA_BASEURL + "/api/" + ACTINIA_VERSION
ACTINIA_AUTH = HTTPBasicAuth("actinia-gdi", "actinia-gdi")
sys.path.append(os.path.join(gisbase, "etc", "python"))

# Import the GRASS GIS packages we need.
import grass.script as gs
import grass.jupyter as gj

# Start GRASS Session
## Set your grass data location
gj.init("../actinia-core-data/grassdb", "nc_spm_08", "PERMANENT")

In [11]:
!ace location="triangle_nc" mapset="simwe" script="./scripts/simwe_100yr.sh" -d

{
  "version": "1",
  "list": [
    {
      "module": "g.region",
      "id": "g.region_1804289383",
      "inputs": [
        {
          "param": "vector",
          "value": "counties"
        },
        {
          "param": "align",
          "value": "nlcd_2019_cog"
        }
      ]
    },
    {
      "module": "importer",
      "id": "importer_1804289383",
      "inputs": [
        {
          "import_descr": {
            "source": "https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/LC20_Elev_220_cog.tif",
            "type": "raster"
          },
          "param": "raster",
          "value": "elevation"
        }
      ]
    },
    {
      "module": "importer",
      "id": "importer_1804289383",
      "inputs": [
        {
          "import_descr": {
            "source": "https://storage.cloud.google.com/tomorrownow-actinia-dev/SpatialData/lulc/nlcd/nlcd_to_mannings.txt",
            "type": "vector"
          },
          "param": "vector",
          "value

In [41]:
import uuid
PCHAIN = !ace location="triangle_nc" mapset="simwe1" script="./scripts/simwe_100yr.sh" -d
pchain = json.loads(str("".join(PCHAIN)))
# # print_as_json(pchain)
# pchain['list'][0]['inputs'][2]['comment'] = "example inpute: geoid in ('48113','48397','48085','48439','48121')"
# pchain['list'][0]['inputs'][2]['value'] = "geoid in {{counties}}"
    

observered_pop = {
    "import_descr": {
        "source": "https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/lulc/nlcd/nlcd_to_mannings.txt",
        "type": "file"
    },
    "param": "rules",
    "value": "$file::nlcd_to_mannings.txt"
}
    
# projected_pop = {
#     "import_descr": {
#         "source": "https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/Hauer_2020_2100_counties_SSP2_projections_demand.csv",
#         "type": "file"
#     },
#     "param": "csv",
#     "value": "$file::projected_population.csv"
# }

importer = {
    "id": "importer_1804289383",
    "module": "importer",
    "inputs": [
        observered_pop
        # projected_pop
    ],
    "outputs": [
        {
            "export": {"type": "file", "format": "TXT"},
            "param": "rules",
            "value": "$file::nlcd_to_mannings"
        }
    #     {
    #         "export": {"type": "file", "format": "CSV"},
    #         "param": "patch_sizes",
    #         "value": "$file::patches"
    #     },
    #     {
    #         "export": {"type": "file", "format": "CSV"},
    #         "param": "output",
    #         "value": "$file::potential"
    #     }
    ]
}

updated_list = [importer] + pchain['list'] 
pchain['list'] = updated_list

url =  "http://localhost:8088/api/v3/locations/triangle_nc/mapsets/simwe1/processing_async"
r = requests.post(
            url, 
            auth=ACTINIA_AUTH,
            json=pchain,
            headers={"content-type": "application/json; charset=utf-8"}
        )
jsonResponse = r.json()
print(f"Response: {r.json()}")

Response: {'accept_datetime': '2023-03-03 03:26:03.483524', 'accept_timestamp': 1677813963.483522, 'api_info': {'endpoint': 'asyncpersistentresource', 'method': 'POST', 'path': '/api/v3/locations/triangle_nc/mapsets/simwe1/processing_async', 'request_url': 'http://localhost:8088/api/v3/locations/triangle_nc/mapsets/simwe1/processing_async'}, 'datetime': '2023-03-03 03:26:03.485430', 'http_code': 200, 'message': 'Resource accepted', 'process_chain_list': [], 'process_results': {}, 'queue': 'local', 'resource_id': 'resource_id-f3578c49-d4b9-42a5-95c5-b6ae8c45633e', 'status': 'accepted', 'time_delta': 0.0019154548645019531, 'timestamp': 1677813963.4854286, 'urls': {'resources': [], 'status': 'http://localhost:8088/api/v3/resources/actinia-gdi/resource_id-f3578c49-d4b9-42a5-95c5-b6ae8c45633e'}, 'user_id': 'actinia-gdi'}


In [42]:
RESPONSE_ID = jsonResponse['urls']['status']
!curl -u 'actinia-gdi:actinia-gdi' -X GET $RESPONSE_ID | jq

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4051  100  4051    0     0   1164      0  0:00:03  0:00:03 --:--:--  1164
{
  "accept_datetime": "2023-03-03 03:26:03.483524",
  "accept_timestamp": 1677813963.483522,
  "api_info": {
    "endpoint": "asyncpersistentresource",
    "method": "POST",
    "path": "/api/v3/locations/triangle_nc/mapsets/simwe1/processing_async",
    "request_url": "http://localhost:8088/api/v3/locations/triangle_nc/mapsets/simwe1/processing_async"
  },
  "datetime": "2023-03-03 03:26:04.695787",
  "exception": {
    "message": "AsyncProcessError:  Error while running executable <r.recode>",
    "traceback": [
      "  File \"/usr/lib/python3.9/site-packages/actinia_core/processing/actinia_processing/ephemeral_processing.py\", line 1930, in run\n    self._execute()\n",
      "  File \"/usr/lib/python3.9/site-packages/actinia_core/processing/actinia

In [43]:
!ace location="triangle_nc" mapset="simwe" script="./scripts/simwe_100yr.sh"

http://localhost:8088/api/v3/locations/triangle_nc/mapsets/simwe/processing_async
{'version': '1', 'list': [{'module': 'g.region', 'id': 'g.region_1804289383', 'inputs': [{'param': 'vector', 'value': 'counties'}, {'param': 'align', 'value': 'nlcd_2019_cog'}]}, {'module': 'importer', 'id': 'importer_1804289383', 'inputs': [{'import_descr': {'source': 'https://storage.googleapis.com/tomorrownow-actinia-dev/SpatialData/LC20_Elev_220_cog.tif', 'type': 'raster'}, 'param': 'raster', 'value': 'elevation'}]}, {'module': 'r.slope.aspect', 'id': 'r.slope.aspect_1804289383', 'inputs': [{'param': 'elevation', 'value': 'elevation'}, {'param': 'format', 'value': 'degrees'}, {'param': 'precision', 'value': 'FCELL'}, {'param': 'zscale', 'value': '1.0'}, {'param': 'min_slope', 'value': '0.0'}, {'param': 'nprocs', 'value': '1'}, {'param': 'memory', 'value': '300'}], 'outputs': [{'param': 'dx', 'value': 'dx'}, {'param': 'dy', 'value': 'dy'}]}, {'module': 'r.sim.water', 'id': 'r.sim.water_1804289383', 'in